# Creación y conexión a la base de datos

In [1]:
# Traer SQLite
import sqlite3
import pandas as pd

In [2]:
# crear y conectar a la base de datos (Ojo, si no existe, la crea, si existe sólo se conecta).

conn = sqlite3.connect('lesson.db')

In [3]:
type(conn)

sqlite3.Connection

In [4]:
# conexión cerrada
conn.close()

In [5]:
with sqlite3.connect('lesson.db') as conn:
  # espacio para utilizar conn
  pass

# Construyendo mi base de datos: Usuarios y comentarios

In [6]:
# Creo mi cursor para poder operar con mi conexión de SQLite3
with sqlite3.connect('lesson.db') as conn:
  cursor = conn.cursor();

In [7]:
# crear tabla
cursor.execute("""CREATE TABLE IF NOT EXISTS 
User(
  email text, 
  first_name text, 
  last_name text, 
  address text, 
  age integer, 
  PRIMARY KEY (email)
)
 """)

In [9]:
cursor.execute("INSERT INTO user VALUES ('bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 'de')")

IntegrityError: ignored

In [10]:
cursor.execute("INSERT INTO user VALUES ('tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39)")

In [11]:
# revisar mis tablas
tabla_de_sqlite3 = cursor.execute("SELECT * FROM sqlite_master WHERE type='table'")

for tupla in tabla_de_sqlite3:
  print(tupla)

('table', 'User', 'User', 2, 'CREATE TABLE User(\n  email text, \n  first_name text, \n  last_name text, \n  address text, \n  age integer, \n  PRIMARY KEY (email)\n)')


In [12]:
# revisar mis tablas (nombre)
tabla_de_sqlite3 = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

for tupla in tabla_de_sqlite3:
  print(tupla)

('User',)


In [13]:
# guardar los inserts
conn.commit()

In [14]:
# traer usuarios
tabla_usuarios = cursor.execute("SELECT * FROM user")

tabla_usuarios

In [15]:
for tupla in tabla_usuarios:
  print(tupla)

('bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 'de')
('tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39)


In [17]:
rows = cursor.execute("SELECT email, first_name FROM user")

for row in rows:
    print(row)

('bob@example.com', 'Bob')
('tom@web.com', 'Tom')


In [18]:
rows = cursor.execute("SELECT first_name, last_name FROM user")

for row in rows:
    print(row)

('Bob', 'Codd')
('Tom', 'Fake')


In [22]:
rows = cursor.execute("SELECT * FROM user WHERE age = 39")
for row in rows:
    print(row)

('tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39)


In [23]:
rows = cursor.execute("SELECT email, age FROM user ORDER BY age DESC")

for row in rows:
    print(row)

('bob@example.com', 'de')
('tom@web.com', 39)


In [24]:
cursor.execute('ALTER TABLE user ADD COLUMN gender text')

In [25]:
conn.commit()

In [26]:
tabla_usuarios = cursor.execute("SELECT email, gender FROM user")

for tupla in tabla_usuarios:
  print(tupla)

('bob@example.com', None)
('tom@web.com', None)


In [27]:
cursor.execute("UPDATE user SET gender='M'")
conn.commit()

In [28]:
tabla_usuarios = cursor.execute("SELECT email, gender FROM user")

for tupla in tabla_usuarios:
  print(tupla)

('bob@example.com', 'M')
('tom@web.com', 'M')


In [29]:
cursor.execute("UPDATE user SET age=38 WHERE user.email='bob@example.com'")
conn.commit()

In [30]:
tabla_usuarios = cursor.execute("SELECT email, gender, age FROM user")

for tupla in tabla_usuarios:
  print(tupla)

('bob@example.com', 'M', 38)
('tom@web.com', 'M', 39)


In [31]:
rows = cursor.execute("SELECT email, gender, age FROM user ORDER BY age DESC")

for row in rows:
    print(row)

('tom@web.com', 'M', 39)
('bob@example.com', 'M', 38)


In [32]:
cursor.execute("INSERT INTO user VALUES('shelly@www.com', 'Shelly', 'Milar', '123, Ocean View Lane', 39, 'F')")
conn.commit()

In [33]:
tabla_usuarios = cursor.execute("SELECT * FROM user")

for tupla in tabla_usuarios:
  print(tupla)

('bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 38, 'M')
('tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39, 'M')
('shelly@www.com', 'Shelly', 'Milar', '123, Ocean View Lane', 39, 'F')


In [ ]:
pd.read_sql_query("SELECT * FROM user", conn)

,email,first_name,last_name,address,age,gender
0,bob@example.com,Bob,Codd,"123 Fantasy lane, Fantasy City",38,M
1,tom@web.com,Tom,Fake,"456 Fantasy lane, Fantasy City",39,M
2,shelly@www.com,Shelly,Milar,"123, Ocean View Lane",39,F


# Crear relación con otra tabla

In [34]:
# para crear una tabla que contenga una llave foranea es siempre necesario activar la funcionalidad
cursor.execute('PRAGMA foreign_keys = 1')

In [35]:
# Aquí vamos a crear nuestra nueva tabla
sql = """
    CREATE TABLE IF NOT EXISTS comments(
        user_id text,
        comments text,
        FOREIGN KEY (user_id) REFERENCES user (email)
        ON DELETE CASCADE ON UPDATE NO ACTION
    )
"""
cursor.execute(sql)

In [36]:
tabla_de_sqlite3 = cursor.execute("SELECT * FROM sqlite_master WHERE type='table'")

for tupla in tabla_de_sqlite3:
  print(tupla)

('table', 'User', 'User', 2, 'CREATE TABLE User(\n  email text, \n  first_name text, \n  last_name text, \n  address text, \n  age integer, gender text, \n  PRIMARY KEY (email)\n)')
('table', 'comments', 'comments', 4, 'CREATE TABLE comments(\n        user_id text,\n        comments text,\n        FOREIGN KEY (user_id) REFERENCES user (email)\n        ON DELETE CASCADE ON UPDATE NO ACTION\n    )')


In [37]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,User,User,2,"CREATE TABLE User(\n email text, \n first_na..."
1,table,comments,comments,4,"CREATE TABLE comments(\n user_id text,\..."


In [38]:
sql = "INSERT INTO comments VALUES ('{}', '{}')"
rows = cursor.execute('SELECT * FROM user ORDER BY age')
for row in rows:
    email = row[0]
    print("Going to create rows for {}".format(email))
    name = row[1] + " " + row[2]
    for i in range(10):
        comment = "This is comment {} by {}".format(i, name)
        conn.cursor().execute(sql.format(email, comment))
        
conn.commit()

Going to create rows for bob@example.com
Going to create rows for tom@web.com
Going to create rows for shelly@www.com


In [39]:
tabla_comments = cursor.execute("SELECT * FROM comments")

for tupla in tabla_comments:
  print(tupla)

('bob@example.com', 'This is comment 0 by Bob Codd')
('bob@example.com', 'This is comment 1 by Bob Codd')
('bob@example.com', 'This is comment 2 by Bob Codd')
('bob@example.com', 'This is comment 3 by Bob Codd')
('bob@example.com', 'This is comment 4 by Bob Codd')
('bob@example.com', 'This is comment 5 by Bob Codd')
('bob@example.com', 'This is comment 6 by Bob Codd')
('bob@example.com', 'This is comment 7 by Bob Codd')
('bob@example.com', 'This is comment 8 by Bob Codd')
('bob@example.com', 'This is comment 9 by Bob Codd')
('tom@web.com', 'This is comment 0 by Tom Fake')
('tom@web.com', 'This is comment 1 by Tom Fake')
('tom@web.com', 'This is comment 2 by Tom Fake')
('tom@web.com', 'This is comment 3 by Tom Fake')
('tom@web.com', 'This is comment 4 by Tom Fake')
('tom@web.com', 'This is comment 5 by Tom Fake')
('tom@web.com', 'This is comment 6 by Tom Fake')
('tom@web.com', 'This is comment 7 by Tom Fake')
('tom@web.com', 'This is comment 8 by Tom Fake')
('tom@web.com', 'This is comm

In [ ]:
sql = """
  SELECT * FROM comments JOIN user
  ON comments.user_id = user.email
  WHERE user.email = 'tom@web.com'
"""

join_table = cursor.execute(sql)

for tupla in join_table:
  print(tupla)

('tom@web.com', 'This is comment 0 by Tom Fake', 'tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39, 'M')
('tom@web.com', 'This is comment 1 by Tom Fake', 'tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39, 'M')
('tom@web.com', 'This is comment 2 by Tom Fake', 'tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39, 'M')
('tom@web.com', 'This is comment 3 by Tom Fake', 'tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39, 'M')
('tom@web.com', 'This is comment 4 by Tom Fake', 'tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39, 'M')
('tom@web.com', 'This is comment 5 by Tom Fake', 'tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39, 'M')
('tom@web.com', 'This is comment 6 by Tom Fake', 'tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39, 'M')
('tom@web.com', 'This is comment 7 by Tom Fake', 'tom@web.com', 'Tom', 'Fake', '456 Fantasy lane, Fantasy City', 39, 'M')
('tom@web.com', 'This is

In [40]:
sql = """
  SELECT * FROM comments JOIN user
  ON comments.user_id = user.email
"""

join_table = cursor.execute(sql)

for tupla in join_table:
  print(tupla)

('bob@example.com', 'This is comment 0 by Bob Codd', 'bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 38, 'M')
('bob@example.com', 'This is comment 1 by Bob Codd', 'bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 38, 'M')
('bob@example.com', 'This is comment 2 by Bob Codd', 'bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 38, 'M')
('bob@example.com', 'This is comment 3 by Bob Codd', 'bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 38, 'M')
('bob@example.com', 'This is comment 4 by Bob Codd', 'bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 38, 'M')
('bob@example.com', 'This is comment 5 by Bob Codd', 'bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 38, 'M')
('bob@example.com', 'This is comment 6 by Bob Codd', 'bob@example.com', 'Bob', 'Codd', '123 Fantasy lane, Fantasy City', 38, 'M')
('bob@example.com', 'This is comment 7 by Bob Codd', 'bob@example.com', 'Bob', 'Codd', '12

In [41]:
table = cursor.execute("SELECT COUNT(*) FROM comments")

for tupla in table:
  print(tupla)

(30,)


In [42]:
table = cursor.execute("SELECT AVG(age) FROM user")

for tupla in table:
  print(tupla)

(38.666666666666664,)


In [44]:
table = cursor.execute("SELECT AVG(email) FROM user")

for tupla in table:
  print(tupla)

(38,)
